# Setup

In [1]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import cv2
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from geopandas import GeoDataFrame
from shapely.geometry import Point
from shapely.geometry import box

from analyser import road_shapefile_to_opencv_mat, distance_transform_scipy

## Paths

In [26]:
PRESENTATION_MEDIA_DIR = Path("presentation_media")
DATA_DIR = Path("data/processed")

# Process Timeseries

In [ ]:
data_dir = DATA_DIR / "2024-09-26/PI4"

for file_path in data_dir.glob("*.csv"):
    df = pd.read_csv(file_path)
    if df["CO2"].std() < 50:
        plt.plot(df["CO2"])
    plt.ylim(400, 600)

plt.show()

# Process Locations

In [ ]:
dir_path = Path("data/processed/2024-09-26/locations")
locations_file_path = Path("data/processed/2024-09-26/locations/locations.csv")
df = pd.read_csv(locations_file_path)

df["geometry"] = df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")  # Setting CRS to WGS84
gdf.to_file(dir_path / "locations.shp")

gdf.plot()

# Parameters

## Distance Transform

In [ ]:
proj_name = "RoadSectionLine"
proj_path = Path("gis/RoadSectionLine_Jul2024")
shp_file_path = proj_path / f"{proj_name}.shp"

gdf: GeoDataFrame = gpd.read_file(shp_file_path)
gdf.plot()

In [ ]:
min_lon, max_lon = 103.76, 103.775
min_lat, max_lat = 1.304, 1.325

bbox = box(min_lon, min_lat, max_lon, max_lat)
bbox_gdf = gpd.GeoDataFrame([[1]], geometry=[bbox], crs="EPSG:4326")
transformed_gdf = gdf.to_crs(crs="EPSG:4326")
cropped_gdf = gpd.overlay(
    transformed_gdf, bbox_gdf, how="intersection", keep_geom_type=False
)
cropped_gdf.plot()

points_gdf: GeoDataFrame = gpd.read_file(
    Path("data/processed/2024-09-26/locations/locations.shp")
)
points_gdf.plot(ax=plt.gca(), color="r")

In [ ]:
road_image = road_shapefile_to_opencv_mat(geo_data=cropped_gdf, img_width=3200)
dist = cv2.distanceTransform(cv2.bitwise_not(road_image), cv2.DIST_L2, 0)

plt.imshow(dist[:, :], origin="lower")
plt.colorbar()
plt.show()

In [ ]:
transform, distance_transform = distance_transform_scipy(
    gdf=cropped_gdf, resolution=1e-5
)

with rasterio.open(
    proj_path / "processed" / "distance_transform.tif",
    "w",
    driver="GTiff",
    height=distance_transform.shape[0],
    width=distance_transform.shape[1],
    count=1,
    dtype=distance_transform.dtype,
    crs=cropped_gdf.crs,
    transform=transform,
) as dst:
    dst.write(distance_transform, 1)

plt.imshow(distance_transform)
plt.colorbar()
plt.show()